# Demo for LumiSpy package working with AttoLight SEM CL data

This notebook shows:

- Loading a `HYPCard.sur` file correctly with the lumispy load function
- Plotting cathodoluminescence data in different ways
- Dealing with metadata
- Correcting for spectral defects

Import packages:

In [ ]:
%matplotlib widget
# Use '%matplotlib widget' in JupterLab and '%matplotlib notebook' in JupyterNotebook for interactive inline functionality (e.g. on binder)
#For pop-up window plots on your local computer, use '%matplotlib tk' or '%matplotlib qt' instead
import hyperspy.api as hs
import lumispy as lum
import os, glob

## Loading HYPCard files

Use the `hs.load()` function with the `signal_type` class for AttoLight CL-SEM specific data (`CL_SEM`).
State the relative path to the `HYPCard.sur` file.

*You can also leave the path empty. A pop-up window will appear to select the `HYPCard.sur` file from the browser.*

In [ ]:
root_folder = "demo-files/load_from_AttoLightSEM/"
path = os.path.join(root_folder, 'HYPCard.sur')

cl = hs.load(path, signal_type='CL_SEM',)
cl

In [ ]:
cl.plot()

## Metadata

The `.sur` file contains the general Hyperspy-specific metadata (`cl.metadata`).
A more detailed metadata is also available via `cl.original_metadata`:

In [ ]:
cl.metadata

In [ ]:
cl.original_metadata.Object_0_Channel_0.Parsed

## Data pre-processing

Let's start some pre-processing methods:

### Background removal

In the AttoLight system, if a background is taken before mapping, the background is stored automatically in the signal folder.

Otherwise, manually load a background file, using `np.loadtxt(path_to_file)`.

In [ ]:
try:
    bkg_path = os.path.join(root_folder, '*.txt')
    bkg_path = glob.glob(bkg_path)[0]
except IndexError:
    print('Please, specify a `background.txt` file path below:')
    bkg_path = ''

In [ ]:
import numpy as np
cl_bkg = np.loadtxt(bkg_path)[1]
print(cl_bkg.shape)

# Run background subtraction
cl = cl - cl_bkg

### Correction of acquisition defects

Correct for the intrinsic shift caused by the misalignment of the grating with the spectrometer aperture centre.

In [ ]:
calibration_factor = 131072
grating = int(cl.original_metadata.Object_0_Channel_0.Parsed.SPECTROMETER.Grating__Groove_Density)

if grating == 150:
    correction_factor_grating = 2.73E-04 # 150 gr/mm grating
elif grating == 600:
    correction_factor_grating = 6.693659836087227e-05 # 600 gr/mm grating
else:
    raise ImportError('Grating correction not available')

fov = cl.original_metadata.Object_0_Channel_0.Parsed.SITE_IMAGE.Field_of_view *1e6

grating_calibrations = {
    'cal_factor' : calibration_factor,
    'corr_factor_grating' : correction_factor_grating,
    'field_of_view_um' : fov,
}

In [ ]:
cl.correct_grating_shift(*grating_calibrations.values())

The edges of the scan get higher intensities, so they can be cropped. 
If you set the `inplace` parameter to `True` the original CLSEMSpectrum object will be modified, if `False` a cropped copy of it will be created.

In [ ]:
cl = cl.crop_edges(crop_px=5)

Remove the cosmic ray saturated pixels (pixels with sharp spikes):

In [ ]:
cl.remove_spikes(inplace=True)

If not all spikes are removed, you can use the GUI to manually select pixels by calling `cl.remove_spikes(interactive=True, inplace=True)`

## Plotting data

Plot the corrected data:

In [ ]:
cl.plot()
cl.mean().plot()

## Save the data as `.hspy` format

Look out for other `lumispy-demos` notebooks to find examples on how to analyse and fit luminescence data.
Many notebooks start directly with a `.hspy` file format.

In [ ]:
# cl.save('path_to_save.hspy')

